@title : headline
@text : article

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
from __future__ import print_function


import json
import os
import pandas
import io
import sys
import re
import tqdm
from tqdm import tqdm


class ProgressBar(object):
    DEFAULT = 'Progress: %(bar)s %(percent)3d%%'
    FULL = '%(bar)s %(current)d/%(total)d (%(percent)3d%%) %(remaining)d to go'

    def __init__(self, total, width=40, fmt=DEFAULT, symbol='=',
                 output=sys.stderr):
        assert len(symbol) == 1

        self.total = total
        self.width = width
        self.symbol = symbol
        self.output = output
        self.fmt = re.sub(r'(?P%\(.+?\))d',
            r'\g%dd' % len(str(total)), fmt)

        self.current = 0

    def __call__(self):
        percent = self.current / float(self.total)
        size = int(self.width * percent)
        remaining = self.total - self.current
        bar = '[' + self.symbol * size + ' ' * (self.width - size) + ']'

        args = {
            'total': self.total,
            'bar': bar,
            'current': self.current,
            'percent': percent * 100,
            'remaining': remaining
        }
        print('\r' + self.fmt % args, file=self.output, end='')

    def done(self):
        self.current = self.total
        self()
        print('', file=self.output)
default_path = "drive/My Drive/hindi_dataset/"

#-------------------------------save/load--------------------------------------#
pickle_path = default_path + "pickles/"

import pickle

def save(obj , filename):
  print("saving {} ..".format(filename))
  with open(filename, 'wb') as handle:
      pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)
      
def load(filename):
  print("loading {} ..".format(filename))
  with open(filename, 'rb') as handle:
    return pickle.load(handle)
#-----------------------------------------------------------------------------------#  

### Word Embeddings

In [3]:
default_path = "drive/My Drive/hindi_dataset/"
reviews_csv = default_path + "train.csv"

In [4]:
import pandas as pd
reviews = pd.read_csv(reviews_csv)
reviews.head()

,headline,article
0,EXCLUSIVE: दिल्ली में डीजल टैक्सियों पर बैन से...,दिल्ली में सुप्रीम कोर्ट के डीज़ल टैक्सियों को...
1,जॉर्डन: राष्ट्रपति मुखर्जी ने 86 करोड़ डॉलर के...,जॉर्डन के ऐतिहासिक दौरे पर पहुंचे राष्ट्रपति प...
2,UN में पाकिस्तान की राजदूत मलीहा लोधी ने कराई ...,पाकिस्तानी नेताओं को विवादित और हास्यास्पद बया...
3,38 देशों में पीएम नरेंद्र मोदी बायोपिक को रिली...,पीएम नरेंद्र मोदी बायोपिक में विवेक ओबेरॉय ने ...
4,13 अगस्त 2011: दिनभर की बड़ी खबरें पढ़ें,"देश, दुनिया, महानगर, खेल, आर्थिक और बॉलीवुड मे..."


In [5]:

print(len(reviews["article"]))
print(len(reviews["headline"]))

266607
266607


In [8]:
sentence_list = []
count = 0
for doc in tqdm(reviews["article"]):
  count += 1
  if count > 100000:
    break
  sentence_list.append(str(doc).split())
count = 0
for doc in tqdm(reviews["headline"]):
  count += 1
  if count > 100000:
    break
  sentence_list.append(str(doc).split())


 38%|███▊      | 100000/266607 [00:01<00:03, 50907.18it/s]


In [9]:
print(sentence_list[0])

['दिल्ली', 'में', 'सुप्रीम', 'कोर्ट', 'के', 'डीज़ल', 'टैक्सियों', 'को', 'बंद', 'करने', 'के', 'फैसले', 'के', 'बाद', 'हजारों', 'टैक्सी', 'ड्राइवरों', 'की', 'रोजी', 'रोटी', 'पर', 'तो', 'असर', 'पड़ा', 'ही', 'है,', 'लेकिन', 'अब', 'दिल्ली', 'पर', 'एक', 'और', 'नई', 'मुसीबत', 'आ', 'गई', 'है.', 'चुनाव', 'आयोग', 'राजधानी', 'के', '13', 'वार्ड', 'में', 'उपचुनाव', 'करवा', 'रहा', 'है,', 'लेकिन', 'चुनावों', 'से', 'दो', 'हफ्ते', 'पहले', 'चुनाव', 'आयोग', 'में', 'कामकाज', 'ठप्प', 'हो', 'गया', 'है.', 'कमीशन', 'ने', 'किराए', 'पर', 'ली', 'थी', 'डीजल', 'गाड़ियां', 'दरअसल', 'कमीशन', 'ने', 'लगभग', 'सौ', 'गाड़ियां', 'चुनाव', 'के', 'कामकाज', 'को', 'करने', 'के', 'लिए', 'किराए', 'पर', 'लीं,', 'जिनमें', 'सभी', 'डीज़ल', 'से', 'चलने', 'वाली', 'टैक्सी', 'थी.', 'इन्हीं', 'टैक्सियों', 'से', 'चुनाव', 'अधिकारी', 'से', 'लेकर', 'चुनावों', 'का', 'जिम्मा', 'संभालने', 'वाले', 'बाकी', 'कर्मचारी', 'भी', 'एक', 'जगह', 'से', 'दूसरी', 'जगह', 'आते', 'जाते', 'थे.', 'अचानक', 'चुनावों', 'से', 'ठीक', 'पहले', 'आई', 'इस', 'परेशानी', 'ने',

In [10]:
import gensim
model_arabic_vec = gensim.models.Word2Vec(
        sentence_list,
        size=150,
        window=10,
        min_count=2,
        workers=10,
        batch_words=500)
model_arabic_vec.train(sentence_list, total_examples=len(sentence_list), epochs=10)
model_arabic_vec.wv.save(default_path +"model_hindi.model")

In [12]:
model_arabic_vec.wv.save('drive/My Drive/hindi_dataset/pickles/' +"model_arabic_extreme.model")

In [13]:
from gensim.models import KeyedVectors
wv = KeyedVectors.load(default_path +"model_hindi.model", mmap='r')

In [14]:
wv.most_similar(positive = "से")

[('उससे', 0.6852455139160156),
 ('इनसे', 0.5954462885856628),
 ('उनसे', 0.5780140161514282),
 ('इससे', 0.5691067576408386),
 ('में', 0.5070422887802124),
 ('धीरे-धीरे', 0.4515412449836731),
 ('आपसे', 0.44314026832580566),
 ('को', 0.4358074367046356),
 ('ज़ीने', 0.42865630984306335),
 ('Millennium', 0.4282725155353546)]

In [17]:
# प्याज़
wv.most_similar(positive = "में")

[('मे', 0.5442017316818237),
 ('से', 0.5070421695709229),
 ('के', 0.5022450685501099),
 ('finishing', 0.47895705699920654),
 ('her..', 0.4770951271057129),
 ('भी', 0.4733871817588806),
 ('इसमें', 0.4727116823196411),
 ('टेकरी,', 0.45266634225845337),
 ('Pehlu', 0.44873785972595215),
 ('की', 0.44290152192115784)]

In [6]:
vocab_set = set()
clean_documents = []
print("loading documents...")
#progress = ProgressBar(len(documents), fmt=ProgressBar.FULL)
for doc in reviews["article"]:
  vocab_set.update(str(doc).split())
  #clean_documents.append(str(doc).split())
  #progress.current += 1
  #progress()
#progress.done()
#save(clean_documents , "clean_documents_ketab.pkl")


clean_summary = []
print("loading summaries...")
#progress = ProgressBar(len(summary), fmt=ProgressBar.FULL)
for doc in reviews["headline"]:
  vocab_set.update(str(doc).split())
  #clean_summary.append(str(doc).split())
  #progress.current += 1
  #progress()
#progress.done()
#save(clean_summary , "clean_summary_ketab.pkl")

loading documents...
loading summaries...


In [7]:
len(vocab_set)

1164174

In [ ]:
vocab_set

In [10]:
print(clean_documents[0])

['दिल्ली', 'में', 'सुप्रीम', 'कोर्ट', 'के', 'डीज़ल', 'टैक्सियों', 'को', 'बंद', 'करने', 'के', 'फैसले', 'के', 'बाद', 'हजारों', 'टैक्सी', 'ड्राइवरों', 'की', 'रोजी', 'रोटी', 'पर', 'तो', 'असर', 'पड़ा', 'ही', 'है,', 'लेकिन', 'अब', 'दिल्ली', 'पर', 'एक', 'और', 'नई', 'मुसीबत', 'आ', 'गई', 'है.', 'चुनाव', 'आयोग', 'राजधानी', 'के', '13', 'वार्ड', 'में', 'उपचुनाव', 'करवा', 'रहा', 'है,', 'लेकिन', 'चुनावों', 'से', 'दो', 'हफ्ते', 'पहले', 'चुनाव', 'आयोग', 'में', 'कामकाज', 'ठप्प', 'हो', 'गया', 'है.', 'कमीशन', 'ने', 'किराए', 'पर', 'ली', 'थी', 'डीजल', 'गाड़ियां', 'दरअसल', 'कमीशन', 'ने', 'लगभग', 'सौ', 'गाड़ियां', 'चुनाव', 'के', 'कामकाज', 'को', 'करने', 'के', 'लिए', 'किराए', 'पर', 'लीं,', 'जिनमें', 'सभी', 'डीज़ल', 'से', 'चलने', 'वाली', 'टैक्सी', 'थी.', 'इन्हीं', 'टैक्सियों', 'से', 'चुनाव', 'अधिकारी', 'से', 'लेकर', 'चुनावों', 'का', 'जिम्मा', 'संभालने', 'वाले', 'बाकी', 'कर्मचारी', 'भी', 'एक', 'जगह', 'से', 'दूसरी', 'जगह', 'आते', 'जाते', 'थे.', 'अचानक', 'चुनावों', 'से', 'ठीक', 'पहले', 'आई', 'इस', 'परेशानी', 'ने',

In [11]:
print(clean_summary[0])

['EXCLUSIVE:', 'दिल्ली', 'में', 'डीजल', 'टैक्सियों', 'पर', 'बैन', 'से', 'मुश्किल', 'में', 'पड़ा', 'चुनाव', 'आयोग']


In [12]:
clean_documents_list = clean_documents + clean_summary

In [13]:
len(clean_documents_list)

533214

In [14]:
len(clean_documents_list[0][10])

2

In [ ]:
default_path

In [ ]:
model_arabic_vec.wv.save('drive/My Drive/Colab Notebooks/Model 4_5/' +"model_arabic_extreme.model")

In [ ]:
from gensim.models import KeyedVectors
wv = KeyedVectors.load(default_path +"model_hindi.model", mmap='r')

In [ ]:
wv.most_similar(positive = "से")

### Vocab 

In [11]:
!pip install nltk
import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [18]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
from nltk.tokenize import word_tokenize
import collections
import tqdm

def build_dict(train_article_list, train_headline_list, VOCAB_SIZE):
    vocab_counter = collections.Counter()
    for sentence in train_article_list :
        words = list()
        for word in word_tokenize(sentence):
            words.append(word)
        vocab_counter.update(words)
    for sentence in train_headline_list :
        words = list()
        for word in word_tokenize(sentence):
            words.append(word)
        vocab_counter.update(words)
    
    print ("Writing vocab file...")
    with open(os.path.join(pickle_path, "vocab"), 'w', encoding="utf-8") as writer:
      for word, count in vocab_counter.most_common(VOCAB_SIZE):
        writer.write(word + ' ' + str(count) + '\n')
    print ("Finished writing vocab file")
    return 

In [ ]:
import tqdm
from tqdm import tqdm
 
vocab_counter = collections.Counter()
for i in tqdm(reviews.article):
  words = list()
  for word in word_tokenize(i):
      words.append(word)
  vocab_counter.update(words)



for sentence in reviews.headline :
  words = list()
  for word in word_tokenize(sentence):
      words.append(word)
  vocab_counter.update(words)

In [27]:
reviews = reviews.dropna()
reviews = reviews.reset_index(drop=True)
for i in tqdm(reviews.headline):
  words = list()
  for word in word_tokenize(i):
      words.append(word)
  vocab_counter.update(words)

100%|██████████| 266593/266593 [00:51<00:00, 5167.64it/s]


In [29]:
VOCAB_SIZE = 200000
print ("Writing vocab file...")
with open(os.path.join(pickle_path, "vocab"), 'w', encoding="utf-8") as writer:
  for word, count in vocab_counter.most_common(VOCAB_SIZE):
    writer.write(word + ' ' + str(count) + '\n')
print ("Finished writing vocab file")

Writing vocab file...
Finished writing vocab file
